In [1]:
import pickle
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import tensorflow
# Set the seed for current session so that little change occurs on each DL run
tensorflow.random.set_seed(2)
session_conf = tensorflow.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tensorflow.compat.v1.Session(graph=tensorflow.compat.v1.get_default_graph(), config=session_conf)
tensorflow.compat.v1.keras.backend.set_session(sess)

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, GlobalAveragePooling1D, GlobalMaxPooling1D, Concatenate, Dense
from tensorflow.keras.layers import Activation, RepeatVector, Permute, Lambda, Dropout, Multiply, Conv1D, BatchNormalization
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, RobertaTokenizer, TFRobertaModel, BertConfig, TFAlbertModel, AlbertTokenizer

os.chdir("/mnt/batch/tasks/shared/LS_root/mounts/clusters/dscpu/code/Users/Tamilselvan.S/Press Release/Data")

In [18]:
# BERT model and corresponding tokenizer need to be downloaded. How?
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# Once download is complete, save both model and tokenzier. How?
# tokenizer.save_pretrained(path_to_output_directory)
# bert_model.save_pretrained(path_to_output_directory)

# Generate input for BERT, ALBERT and ROBERTA
def create_BERT_input(df, value):
    max_len = 200
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    # tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    # tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
    input_ids, input_masks, input_segments = list(), list(), list()

    # Create BERT input
    for val in (df[value].values):
        tokenized_context = tokenizer.encode_plus(val, add_special_tokens=True, max_length=max_len, pad_to_max_length=True, return_attention_mask=True, return_token_type_ids=True, truncation=True)
        input_ids.append(tokenized_context["input_ids"])
        input_masks.append(tokenized_context["attention_mask"])
        input_segments.append(tokenized_context["token_type_ids"])
    input_ids, input_masks, input_segments = np.asarray(input_ids, dtype=np.int32), np.asarray(input_masks, dtype=np.int32), np.asarray(input_segments, dtype=np.int32)
    print(input_ids.shape, input_masks.shape, input_segments.shape)
    
    return input_ids, input_masks, input_segments


# # Saves the predictions of provided model
# def save_preds(model, epoch):
#     global df_test, X_tfidf_test, input_ids_test, input_masks_test, input_segments_test
#     # Create input for the model
#     preds = model.predict([input_ids_test, input_masks_test, input_segments_test, X_tfidf_test], verbose=1)

#     fp = open("/content/drive/My Drive/JanataHack_IndependenceDay/submit_proba_"+str(epoch)+".csv", "w")
#     fp.write("ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance\n")
#     for id_, pred in zip(df_test["ID"].values, preds):
#     fp.write(str(id_)+","+",".join([str(i) for i in pred])+"\n")
#     fp.close()

#     fp = open("/content/drive/My Drive/JanataHack_IndependenceDay/submit_"+str(epoch)+".csv", "w")
#     fp.write("ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance\n")
#     for id_, pred in zip(df_test["ID"].values, preds):
#     fp.write(str(id_)+","+",".join(["1" if i>=0.5 else "0" for i in pred])+"\n")
#     fp.close()

# Callback to save model's predictions after each epoch
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
      # if epoch%2 == 0 and epoch!=0:
      if epoch!=0:
        save_preds(self.model, epoch)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [19]:
df_train = pd.read_csv("Cleaned_PR_TD.csv")

In [20]:
df_train.head()

Category Date of release  \
0  Corporate Information      2021-04-01   
1  Corporate Information      2021-04-01   
2  Corporate Information      2021-04-02   
3               Offering      2021-04-05   
4  Corporate Information      2021-04-06   

                                    Title / Contents          Issued by  \
0  NTT DATA Ranked No. 8 in Brand Finance's Lates...           NTT DATA   
1  Effective April 1, 2021: itelligence | NTT DAT...           NTT DATA   
2  NTT DATA to publish Digital Thought Leadership...           NTT DATA   
3     Transformer Neural Networks are Shaking up AI   NTT DATA Services   
4  NTT DATA Achieves Key AWS Partner Network Mile...  NTT DATA Services   

  Country Language                                        Description  \
0  Global  English  TOKYO – Apr 1, 2021 – NTT DATA, a global digit...   
1  Global  English  March 30, 2021\n itelligence | NTT DATA Busine...   
2  Global  English  TOKYO – Apr 2, 2021 – NTT DATA, a global digit...   
3      US  English  Scott Boettcher, vice president of data intell...   
4      US  English  PLANO, Texas – April 5, 2021 – NTT DATA Servic...   

                                                 URL Marketing Contact  \
0  https://www.nttdata.com/global/en/media/press-...           Amy Baj   
1  https://us.nttdata.com/en/news/press-release/2...           Amy Baj   
2  https://www.nttdata.com/global/en/media/press-...           Amy Baj   
3  https://searchenterpriseai.techtarget.com/feat...           Amy Baj   
4  https://us.nttdata.com/en/news/press-release/2...           Amy Baj   

  E mail for Marketing Contact  Quarter  Unique Count  Company Issued_Short  \
0          amy.baj@nttdata.com  2021 Q1           NaN      NaN  Global (HQ)   
1          amy.baj@nttdata.com  2021 Q1           NaN      NaN  Global (HQ)   
2          amy.baj@nttdata.com  2021 Q1           NaN      NaN  Global (HQ)   
3          amy.baj@nttdata.com  2021 Q1           NaN      NaN     Services   
4          amy.baj@nttdata.com  2021 Q1           NaN      NaN     Services   

                                   Title / Contents1  \
0              ranked 8 brand finances latest report   
1  effective april 1 2021 itelligence business so...   
2      publish digital thought leadership automotive   
3             transformer neural networks shaking ai   
4        achieves key aws partner network milestones   

                                        Description1  
0  tokyo – apr 1 2021 – global digital business s...  
1  march 30 2021 itelligence business solutions i...  
2  tokyo – apr 2 2021 – global digital business s...  
3  scott boettcher vice president intelligence se...  
4  plano texas – april 5 2021 – services global d...

In [21]:
# Augmented text

df_train = pd.read_csv("Cleaned_PR_TD.csv")

df_tfidf_train = pd.read_csv("best_tfidf.csv")

# Concatenate title and abstract
df_train["Concat"] = df_train["Title / Contents1"].str.cat(df_train["Description1"].str.strip(), sep=" [SEP] ")


# Remove unnecessary shit
df_train["Concat"] = df_train["Concat"].str.strip()


# Lowercase
df_train["Concat"] = df_train["Concat"].str.lower()


# Seems like the data was scraped from a pdf or something cause there are random \n characters in a sentence. Remove them
df_train["Concat"] = df_train["Concat"].str.replace("\\n", " ")

df_train.head()

Category Date of release  \
0  Corporate Information      2021-04-01   
1  Corporate Information      2021-04-01   
2  Corporate Information      2021-04-02   
3               Offering      2021-04-05   
4  Corporate Information      2021-04-06   

                                    Title / Contents          Issued by  \
0  NTT DATA Ranked No. 8 in Brand Finance's Lates...           NTT DATA   
1  Effective April 1, 2021: itelligence | NTT DAT...           NTT DATA   
2  NTT DATA to publish Digital Thought Leadership...           NTT DATA   
3     Transformer Neural Networks are Shaking up AI   NTT DATA Services   
4  NTT DATA Achieves Key AWS Partner Network Mile...  NTT DATA Services   

  Country Language                                        Description  \
0  Global  English  TOKYO – Apr 1, 2021 – NTT DATA, a global digit...   
1  Global  English  March 30, 2021\n itelligence | NTT DATA Busine...   
2  Global  English  TOKYO – Apr 2, 2021 – NTT DATA, a global digit...   
3      US  English  Scott Boettcher, vice president of data intell...   
4      US  English  PLANO, Texas – April 5, 2021 – NTT DATA Servic...   

                                                 URL Marketing Contact  \
0  https://www.nttdata.com/global/en/media/press-...           Amy Baj   
1  https://us.nttdata.com/en/news/press-release/2...           Amy Baj   
2  https://www.nttdata.com/global/en/media/press-...           Amy Baj   
3  https://searchenterpriseai.techtarget.com/feat...           Amy Baj   
4  https://us.nttdata.com/en/news/press-release/2...           Amy Baj   

  E mail for Marketing Contact  Quarter  Unique Count  Company Issued_Short  \
0          amy.baj@nttdata.com  2021 Q1           NaN      NaN  Global (HQ)   
1          amy.baj@nttdata.com  2021 Q1           NaN      NaN  Global (HQ)   
2          amy.baj@nttdata.com  2021 Q1           NaN      NaN  Global (HQ)   
3          amy.baj@nttdata.com  2021 Q1           NaN      NaN     Services   
4          amy.baj@nttdata.com  2021 Q1           NaN      NaN     Services   

                                   Title / Contents1  \
0              ranked 8 brand finances latest report   
1  effective april 1 2021 itelligence business so...   
2      publish digital thought leadership automotive   
3             transformer neural networks shaking ai   
4        achieves key aws partner network milestones   

                                        Description1  \
0  tokyo – apr 1 2021 – global digital business s...   
1  march 30 2021 itelligence business solutions i...   
2  tokyo – apr 2 2021 – global digital business s...   
3  scott boettcher vice president intelligence se...   
4  plano texas – april 5 2021 – services global d...   

                                              Concat  
0  ranked 8 brand finances latest report [sep] to...  
1  effective april 1 2021 itelligence business so...  
2  publish digital thought leadership automotive ...  
3  transformer neural networks shaking ai [sep] s...  
4  achieves key aws partner network milestones [s...

In [22]:
target = pd.get_dummies(df_train["Category"])
df_train = pd.concat([df_train,target], axis = 1)
classes = list(df_train["Category"].value_counts().index)

In [23]:
Y = df_train[classes].values
X_tfidf_train = df_tfidf_train.values[:,1:]
epoch = 12

# Create input for the model
input_ids, input_masks, input_segments = create_BERT_input(df_train, "Concat")

# Load the pretrained model
bert_model = TFBertModel.from_pretrained("bert-base-uncased")
# bert_model = TFRobertaModel.from_pretrained("roberta-base")
# bert_model = TFAlbertModel.from_pretrained("albert-base-v2")

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(507, 200) (507, 200) (507, 200)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### Generate Arch.

In [24]:
# Define the model
input_ids1 = Input(shape=(input_ids.shape[1],), dtype=tensorflow.int32)
token_type_ids = Input(shape=(input_masks.shape[1],), dtype=tensorflow.int32)
attention_mask = Input(shape=(input_segments.shape[1],), dtype=tensorflow.int32)
input3 = Input(shape=(X_tfidf_train.shape[1],), dtype=tensorflow.float32)

# embedding[0] gives token vector. embedding[1] gives pooled vector (single vector representing entire input sequence)
# There are many ways of pooling embedding[0] and combining with embedding[1]. So we are going to try them all out :P
embedding = bert_model(input_ids1, attention_mask=attention_mask, token_type_ids=token_type_ids)

# Additive attention to generate sentence level representation from embedding[0]
attention = Dense(1, activation='tanh')(embedding[0])
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(768)(attention)
attention = Permute([2, 1])(attention)
sent_representation = Multiply()([embedding[0], attention])
sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(768,))(sent_representation)

# Simple Pooling of embedding[0]
x1 = GlobalMaxPooling1D()(embedding[0])
x2 = GlobalAveragePooling1D()(embedding[0])

# Concatenate them all
x1 = Concatenate()([x1, x2, sent_representation, embedding[1]])
x1 = BatchNormalization()(x1)
# x1 = Dropout(0.4)(x1)
x1 = Dense(64, activation="relu")(x1)

# We also concatenate the tfidf scores generated from the Generate_TFIDF_CountVec_Stacked_Scores
x1 = Concatenate()([x1, input3])
x1 = BatchNormalization()(x1)
# x1 = Dropout(0.1)(x1)

# Another way of using embedding[0] is to train a CNN on top of the output token vectors
# 1D CNNs have been reported to work really well
x2 = Conv1D(256, 1)(embedding[0])
x2 = GlobalMaxPooling1D()(x2)

# Finally concatenate everything together
x1 = Concatenate()([x1, x2])
x1 = BatchNormalization()(x1)
# x1 = Dropout(0.2)(x1)
x1 = BatchNormalization()(x1)

# Final output
out = Dense(Y.shape[1], activation="sigmoid")(x1)

model = Model(inputs=[input_ids1, attention_mask, token_type_ids, input3], outputs=out)
model.summary()
# DONT TRAIN THE PRETRAINED MODEL
model.layers[3].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Precision(),
                              tf.keras.metrics.Recall()])
model.fit([input_ids, input_masks, input_segments, X_tfidf_train], Y, batch_size=16, epochs=epoch, validation_split=0.2)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_5 (TFBertModel)   TFBaseModelOutputWit 109482240   input_9[0][0]                    
                                                                 input_11[0][0]             

In [25]:
df_train.shape

(507, 25)

In [ ]:
model.predict()

In [6]:
import pandas as pd
os.chdir("/mnt/batch/tasks/shared/LS_root/mounts/clusters/dscpu/code/Users/Tamilselvan.S/Press Release")
df = pd.read_csv("t&d_fulldata_svm&ovr.csv")

In [7]:
from sklearn.metrics import classification_report

In [8]:
df.head(2)

text                  pred  \
0  csu pueblo   campus covid response judged nati...  Award/Contest Winner   
1  ranks 6 idc fintech rankings enterprise 25 sep...                Others   

                 actual  
0  Award/Contest Winner  
1                Others

In [9]:
print(classification_report(df.actual, df.pred))

                            precision    recall  f1-score   support

AR Report / Market Insight       0.60      0.60      0.60        20
      Award/Contest Winner       0.62      0.87      0.72        15
      Business Partnership       0.80      0.63      0.71        19
     Corporate Information       0.61      0.85      0.71        46
      Merger & Acquisition       1.00      0.67      0.80         9
                  Offering       0.73      0.36      0.48        22
                    Others       0.69      0.45      0.55        20
               Won Project       0.33      0.50      0.40         2

                  accuracy                           0.65       153
                 macro avg       0.67      0.62      0.62       153
              weighted avg       0.68      0.65      0.64       153



In [10]:
df = pd.read_csv("testdata_svmovr.csv")
print(classification_report(df.actual, df.pred))

                            precision    recall  f1-score   support

AR Report / Market Insight       0.58      0.54      0.56        13
      Award/Contest Winner       0.71      0.83      0.77         6
      Business Partnership       0.67      0.50      0.57         8
     Corporate Information       0.56      0.88      0.69        25
      Merger & Acquisition       1.00      0.25      0.40         4
                  Offering       0.78      0.47      0.58        15
                    Others       0.60      0.33      0.43         9
               Won Project       0.50      1.00      0.67         1

                  accuracy                           0.62        81
                 macro avg       0.68      0.60      0.58        81
              weighted avg       0.65      0.62      0.60        81

